In [74]:
import numpy as np
import scipy as sp
import pandas as pd
import time
import sys
import re
import os
import pickle
from sklearn.model_selection import LeaveOneOut
from matplotlib import pyplot as plt
from tqdm import tqdm
from scipy.sparse.linalg import svds
from scipy.optimize import minimize
import scipy.stats as st
from scipy.linalg import qr
import tensorly as tl
import mkl
from statsmodels.regression.linear_model import WLS
mkl.set_num_threads(1)

sys.path.append('../../../package/automl/')
sys.path.append('../4.2_tensor_vs_matrix_completion/')

import util
import multiprocessing as mp
from factorizations import tucker_on_error_tensor

import linalg
import convex_opt
from sklearn.model_selection import LeaveOneOut
import multiprocessing

In [75]:
from matplotlib import rc
import matplotlib.ticker as ticker
rc('font', **{'family': 'serif', 'serif': ['Computer Modern'], 'weight': 'bold'})
rc('text', usetex=True)

In [76]:
def impute_with_coefficients(Y, a, known_indices):
    # find x using matrix division using known portion of a, corresponding columns of A
    x = np.linalg.lstsq(Y[:, known_indices].T, a[:, known_indices].T, rcond=None)[0].T
    # approxim ate full a as x*Y
    return np.dot(x, Y)

In [77]:
def impute_with_coefficients_and_variance(Y, a, σ, known_indices):
    wls = WLS(a[:, known_indices].flatten(), Y[:, known_indices].T, weights=1/σ[known_indices] ** 2)
    r = wls.fit()
    x = r.params.reshape(1, -1)
    # find x using matrix division using known portion of a, corresponding columns of A
#     x = np.linalg.lstsq(Y[:, known_indices].T, a[:, known_indices].T, rcond=None)[0].T
#     X = Y[:, known_indices].T
#     y = a[:, known_indices].T
    # approxim ate full a as x*Y
    return np.dot(x, Y)

In [78]:
# num_cores = multiprocessing.cpu_count()

In [80]:
input_path = <path_for_data>
error_tensor = np.load(os.path.join(input_path, 'error_tensor.npy'))
runtime_tensor = np.load(os.path.join(input_path, 'runtime_tensor.npy'))
with open(os.path.join(input_path, 'configs_tensor.pkl'), 'rb') as handle:
    configs_tensor = pickle.load(handle)
ind_common = [eval(configs_tensor[i, 0, 0, 0, 0, 0])['dataset'] for i in range(configs_tensor.shape[0])]

ranks_for_imputation = (8, 4, 2, 2, 8, 8)

k_dataset_for_factorization = 20
k_estimator_for_factorization = 20

#     initial_runtime_limit_ratio = 5e-5
#     final_runtime_limit_ratio = 1e-3
#     runtime_limit_ratio_step = 5e-5

initial_runtime_limit_ratio = 5e-5
final_runtime_limit_ratio = 5e-4
runtime_limit_ratio_step = 5e-5

columns = ['ratio {}'.format(ratio) for ratio in np.arange(initial_runtime_limit_ratio, 
                                                                               final_runtime_limit_ratio + runtime_limit_ratio_step,
                                                                              runtime_limit_ratio_step)]

In [82]:
loo = LeaveOneOut()

In [83]:
result_path = 'results'

if not os.path.exists(result_path):
    os.makedirs(result_path)

In [84]:
n_procs = 80
n_best = 1
scalarization = 'D'
num_cores = 1
columns = ['ratio {}'.format(ratio) for ratio in np.arange(initial_runtime_limit_ratio, 
                                                                               final_runtime_limit_ratio + runtime_limit_ratio_step,
                                                                              runtime_limit_ratio_step)]

In [85]:
def pivot_columns_time(A, t, t_max, rank=None, columns_to_avoid=None, threshold=None, column_picking_threshold="rank"):
    """
    column_picking_threshold: "rank" or "size".
    """
    if columns_to_avoid is not None:
        set_of_columns_to_avoid = set(columns_to_avoid)
    else:
        set_of_columns_to_avoid = set()
    
    t_sum = 0
    assert (threshold is None) != (rank is None), "Exactly one of threshold and rank should be specified."
    if threshold is not None:
        rank = approx_rank(A, threshold)
    
    if column_picking_threshold == "rank":
        valid = np.where(t <= t_max / (2 * rank))[0]
    elif column_picking_threshold == "size":
        valid = np.where(t <= t_max / (2 * A.shape[1]))[0]
    
    r = []
    i = 0
    if len(valid) < rank: # there aren't enough pipeline to finish in the time budget, then we need to do greedy
        case = 'greedy_initialization'
        idx_sorted_by_time = np.argsort(t)
        while(t_sum < t_max):
            idx = idx_sorted_by_time[i]
            if t_sum + t[idx] <= t_max:
                r.append(idx)
                t_sum = t_sum + t[idx]
                i += 1
            else:
                break
        
    else:
        case = 'qr_initialization'
#     print("{} valid in {}".format(valid, A.shape))
        qr_columns = qr(A[:, valid], pivoting=True)[2]        
        
        while(len(r) < rank):
            qr_column = qr_columns[i]
            if valid[qr_column] not in set_of_columns_to_avoid:
                r.append(valid[qr_column])
                t_sum += t[valid[qr_column]]
            i += 1
    return r, t_sum, case

In [86]:
def approx_rank(a, threshold=0.03):
    """Compute approximate rank of a matrix.

    Args:
        a (np.ndarray):    Matrix for which to compute rank.
        threshold (float): All singular values less than threshold * (largest singular value) will be set to 0
    Returns:
        int: The approximate rank of a.
    """
    s = np.linalg.svd(a, compute_uv=False)
    rank = s[s >= threshold * s[0]]
    return len(rank)

def truncated_pca(a, rank=None, threshold=None):
    """Solves: minimize ||A_XY||^2 where ||.|| is the Frobenius norm.

    Args:
        a (np.ndarray):    Matrix for which to compute PCA.
        threshold (float): Threshold specifying approximate rank of a.
        rank (int):        The approximate rank.
    Returns:
        x, y (np.ndarray): The solutions to the PCA problem.
        vt (np.ndarray):   Transpose of V as specified in the singular value decomposition.
    """
    assert (threshold is None) != (rank is None), "Exactly one of threshold and rank should be specified."
    if threshold is not None:
        rank = approx_rank(a, threshold)
    # std = np.std(a, axis=0)
    u, s, vt = svds(a, k=rank)

    nonzero_pos = np.where(s > 0)[0]
    s = s[nonzero_pos]
    u = u[:, nonzero_pos]
    vt = vt[nonzero_pos, :]

    u = np.fliplr(u)
    s = np.flipud(s)
    vt = np.flipud(vt)
    # sigma_sqrt = np.diag(np.sqrt(s))
    # x = np.dot(u, sigma_sqrt).T
    # # y = np.dot(np.dot(sigma_sqrt, vt), np.diag(std))
    # y = np.dot(sigma_sqrt, vt)

    sigma = np.diag(s)
    x = np.dot(u, sigma).T
    y = vt
    return x, y, vt

Case 1: convexification

In [70]:
def time_limit_solve(t_predicted, t_max, n_cores, Y, scalarization='D'):
    """Solve the following optimization problem:
    minimize -log(det(sum_i v[i]*Y[:, i]*Y[:, i].T)) subject to 0 <= v[i] <= 1 and t_predicted.T * v <= t_max
    The optimal vector v is an approximation of a boolean vector indicating which entries to sample.

    Args:
         t_predicted (np.ndarray): 1-d array specifying predicted runtime for each model setting
         t_max (float):            maximum runtime of sampled model
         n_cores (int):            number of cores to use
         Y (np.ndarray):           matrix representing latent variable weights of error matrix
         scalarization (str):      scalarization method in experimental design.
    Returns:
        np.ndarray:                optimal vector v (not truncated to binary values)
    """

    n = len(t_predicted)

    if scalarization == 'D':
        def objective(v):
            sign, log_det = np.linalg.slogdet(Y @ np.diag(v) @ Y.T)
            return -1 * sign * log_det
    elif scalarization == 'A':
        def objective(v):
            return np.trace(np.linalg.pinv(Y @ np.diag(v) @ Y.T))
    elif scalarization == 'E':
        def objective(v):
            return np.linalg.norm(np.linalg.pinv(Y @ np.diag(v) @ Y.T), ord=2)
    def constraint(v):
        return t_max * n_cores- t_predicted @ v
    v0 = np.full((n, ), 0.5)
    constraints = {'type': 'ineq', 'fun': constraint}
    v_opt = minimize(objective, v0, method='SLSQP', bounds=[(0, 1)] * n, options={'maxiter': 50},
                     constraints=constraints)
    
    return v_opt.x

In [71]:
def get_regret_for_specific_dataset(train_index, test_index):
    print("test index: {}".format(test_index))
    regret = []
    elapsed = []
    idx_to_sample = []
    new_row = np.zeros((1, errmtx_common.shape[1]))    

    X_pca, Y_pca, _ = truncated_pca(errmtx_common[train_index], threshold=0.03)


# NOTE: we are using true running time instead of predicted time for now
    t_predicted = np.array(runtime_df.iloc[0, :])
    t_predicted_sum = np.sum(t_predicted)
    for runtime_limit in np.arange(t_predicted_sum * initial_runtime_limit_ratio, 
                               t_predicted_sum * (final_runtime_limit_ratio + runtime_limit_ratio_step), 
                               t_predicted_sum * runtime_limit_ratio_step):
        start = time.time()
        valid = np.where(t_predicted <= num_cores * runtime_limit/2)[0]
        Y = Y_pca[:, valid]
        v_opt = time_limit_solve(t_predicted=t_predicted[valid], t_max=runtime_limit, n_cores=num_cores, Y=Y, scalarization=scalarization)
        # true best
        y_best_true = min(errmtx_common[test_index[0], :])
        # predicted best; initialize to be 1 (max achievable error)
        
        y_best_pred = 1

        to_sample = []
        total_runtime = 0
        i = 0
        v_opt_argsort_descending = np.argsort(-v_opt)
        while(total_runtime <= runtime_limit):
            idx = valid[v_opt_argsort_descending[i]]
            if total_runtime + t_predicted[idx] < runtime_limit:
                to_sample.append(idx)
                total_runtime = total_runtime + t_predicted[idx]
            i += 1
            if i >= len(valid):
                break
#         print("total runtime: {}, runtime limit: {}".format(total_runtime, runtime_limit))
        idx_to_sample.append(to_sample)
        new_row[:, to_sample] = errmtx_common[test_index, to_sample]
        errmtx_pred[test_index, :] = impute_with_coefficients(Y_pca, new_row, to_sample)
        # predicted best; only update when the new best is better (i.e., has lower error)
        y_best_in_this_round = min(np.concatenate((errmtx_common[test_index[0], to_sample], errmtx_common[test_index[0], np.argsort(errmtx_pred[test_index[0], :])[:n_best]])))        
        y_best_pred = min(y_best_pred, y_best_in_this_round)
        # collect regret
        elapsed.append(time.time() - start)
        regret.append(y_best_pred - y_best_true)
        
    regret_single = pd.DataFrame(np.array(regret).reshape(1, -1), index=[ind_common[test_index[0]]], columns=columns)
    elapsed_single = pd.DataFrame(np.array(elapsed).reshape(1, -1), index=[ind_common[test_index[0]]], columns=columns)
    idx_to_sample_single = pd.DataFrame(np.array(idx_to_sample).reshape(1, -1), index=[ind_common[test_index[0]]], columns=columns)
    return regret_single, idx_to_sample_single, elapsed_single
    

In [72]:
# for train_index, test_index in loo.split(errmtx_common):
#     print(get_regret_for_specific_dataset(train_index, test_index))
#     break

In [73]:
n_procs = 112

regret_all = pd.DataFrame(columns=columns)
elapsed_all = pd.DataFrame(columns=columns)
idx_to_sample_all = pd.DataFrame(columns=columns) 
errors_encountered_all = []

p1 = mp.Pool(n_procs)
r = [p1.apply_async(get_regret_for_specific_dataset, args=[train_index, test_index]) 
          for train_index, test_index in loo.split(errmtx_common)]
p1.close()
p1.join()       
    
for item in r:
    regret_all = regret_all.append(item.get()[0])
    idx_to_sample_all = idx_to_sample_all.append(item.get()[1])
    elapsed_all = elapsed_all.append(item.get()[2])

test index: [0]
test index: [2]
test index: [1]
test index: [3]
test index: [4]
test index: [7]
test index: [10]
test index: [11]
test index: [14]
test index: [17]
test index: [8]
test index: [13]
test index: [15]
test index: [5]
test index: [6]
test index: [16]
test index: [12]
test index: [9]
test index: [26]
test index: [18]
test index: [27]
test index: [23]
test index: [19]
test index: [30]
test index: [22]
test index: [24]
test index: [21]
test index: [25]
test index: [20]
test index: [32]
test index: [28]
test index: [29]
test index: [34]
test index: [36]
test index: [31]
test index: [33]
test index: [39]
test index: [40]
test index: [38]
test index: [43]
test index: [37]
test index: [35]
test index: [45]
test index: [41]
test index: [44]
test index: [47]
test index: [46]
test index: [48]
test index: [49]
test index: [42]
test index: [55]
test index: [53]
test index: [54]
test index: [50]
test index: [56]
test index: [52]
test index: [59]
test index: [51]
test index: [58]
test in

ValueError: unexpected array size: new_size=1, got array with arr_size=0


Case 2: greedy

In [87]:
def greedy_stepwise_selection_with_time(Y, t, initialization, t_elapsed, t_max, idx_to_exclude=None, verbose=False):
    """
    Only D-optimal.
    """
    n_cols = Y.shape[1]
    assert n_cols == len(t), "Dimensionality mismatch!"
    if t_elapsed >= t_max:
        print("time limit exceeded!")
        return []
    
    t_sum = t_elapsed
    X = Y[:, initialization] @ Y[:, initialization].T
    selected = list(initialization)
    
    if idx_to_exclude is not None:
        set_of_indices_to_avoid = set(idx_to_exclude)
    else:
        set_of_indices_to_avoid = set()
    
    while(t_sum <= t_max):
        if verbose and not iteration % 10:
            print("Iteration {}".format(iteration))
        to_select = list(set(range(n_cols)).difference(set(selected)).difference(set_of_indices_to_avoid))
        try:
            X_inv = np.linalg.inv(X)
        except:
            break
        obj_all = np.array([(Y[:, i] @ X_inv @ Y[:, i])/t[i] for i in to_select])
        
        if not all(np.isnan(obj_all)):
            valid = np.where(t_sum + t[to_select] <= t_max)[0]
            if len(valid) == 0:
                break
            to_add = to_select[valid[np.nanargmax(obj_all[valid])]]
        else:
            break        
            
#         print(to_add)

        X = X + np.outer(Y[:, to_add], Y[:, to_add])
        selected.append(to_add)
        t_sum = t_sum + t[to_add]
#         print("total time: {}".format(t_sum))
    
    print(len(selected))
    if verbose:
        print("condition number of final design matrix: {}".format(np.linalg.cond(X)))
    return selected

In [88]:
def get_regret_for_specific_dataset_greedy(train_index, test_index):
    print("test index: {}".format(test_index))
    regret = []
    elapsed = []
    idx_to_sample = []

    # read data
    error_tensor = np.load(os.path.join(input_path, 'error_tensor.npy'))
    runtime_tensor = np.load(os.path.join(input_path, 'runtime_tensor.npy'))
    with open(os.path.join(input_path, 'configs_tensor.pkl'), 'rb') as handle:
        configs_tensor = pickle.load(handle)
    ind_common = [eval(configs_tensor[i, 0, 0, 0, 0, 0])['dataset'] for i in range(configs_tensor.shape[0])]        

    error_tensor_train = error_tensor[train_index]
    _, _, error_tensor_train_imputed, _ = tucker_on_error_tensor(error_tensor_train, 
                                                              ranks_for_imputation, 
                                                              save_results=False, 
                                                              verbose=False)

    core_tr, factors_tr = tl.decomposition.tucker(error_tensor_train_imputed, ranks=(k_dataset_for_factorization, 4, 2, 2, 8, k_estimator_for_factorization))
    pipeline_latent_factors = tl.unfold(tl.tenalg.multi_mode_dot(core_tr, factors_tr[1:], modes=[1, 2, 3, 4, 5]), mode=0)
    U_t, S_t, Vt_t = sp.linalg.svd(pipeline_latent_factors, full_matrices=False)
    Y_pca = Vt_t  

    error_matrix_train = tl.unfold(error_tensor_train_imputed, mode=0)
    error_tensor_test = error_tensor[test_index]
    error_matrix_test = tl.unfold(np.array([error_tensor_test]), mode=0)[0]
    missing_entries_in_test = np.where(np.isnan(error_matrix_test))[0]
    idx_to_exclude = missing_entries_in_test
    errmtx_common = np.insert(error_matrix_train, test_index, error_matrix_test, axis=0)        

    runtime_tensor_test = runtime_tensor[test_index]
    t_predicted = tl.unfold(np.array([runtime_tensor_test]), mode=0)[0]



    t_predicted_sum = np.nansum(t_predicted)
    # NOTE: we are using true running time instead of predicted time for now
    new_row = np.zeros((1, errmtx_common.shape[1]))
    new_row_pred = np.zeros((1, errmtx_common.shape[1]))
    y_best_pred = 1

    for idx, runtime_limit in enumerate(t_predicted_sum * np.arange(initial_runtime_limit_ratio, 
                           final_runtime_limit_ratio + runtime_limit_ratio_step, 
                           runtime_limit_ratio_step)):
        print("on test dataset {}, runtime limit index {}".format(test_index, idx))
        start = time.time()
        valid = np.where(t_predicted <= num_cores * runtime_limit/2)[0]
        Y = Y_pca[:, valid]
        selected_columns_qr, t_sum, case = pivot_columns_time(Y, t_predicted[valid], runtime_limit, 
                                                        columns_to_avoid=idx_to_exclude,
                                                        rank=Y_pca.shape[0])

        # true best
        y_best_true = np.nanmin(errmtx_common[test_index[0], :])
        if case == 'greedy_initialization':
            print(case)
            y_best_pred = np.nanmin([y_best_pred, 
                                   np.nanmin([errmtx_common[test_index[0], selected_columns_qr]])])
            idx_to_sample.append(selected_columns_qr)
        elif case == 'qr_initialization':
            to_sample = greedy_stepwise_selection_with_time(Y=Y,
                                                        t=t_predicted[valid],
                                                        initialization=selected_columns_qr,
                                                        t_elapsed=t_sum,
                                                        t_max=runtime_limit,
                                                        idx_to_exclude=idx_to_exclude)
            
            idx_to_sample.append(to_sample)
            new_row[:, to_sample] = errmtx_common[test_index, to_sample]
            new_row_pred[0, :] = impute_with_coefficients(Y_pca, new_row, to_sample)
            # predicted best; only update when the new best is better (i.e., has lower error)            
            y_best_in_this_round = min(np.concatenate((errmtx_common[test_index[0], to_sample], errmtx_common[test_index[0], np.argsort(new_row_pred[0, :])[:n_best]])))        
            y_best_pred = min(y_best_pred, y_best_in_this_round)
        # collect regret
        elapsed.append(time.time() - start)
        regret.append(y_best_pred - y_best_true)
    regret_single = pd.DataFrame(np.array(regret).reshape(1, -1), index=[ind_common[test_index[0]]], columns=columns)
    elapsed_single = pd.DataFrame(np.array(elapsed).reshape(1, -1), index=[ind_common[test_index[0]]], columns=columns)
    idx_to_sample_single = pd.DataFrame(np.array(idx_to_sample).reshape(1, -1), index=[ind_common[test_index[0]]], columns=columns)
    
    return regret_single, idx_to_sample_single, elapsed_single

In [89]:
n_procs = 112

regret_all_greedy = pd.DataFrame(columns=columns)
idx_to_sample_all_greedy = pd.DataFrame(columns=columns)
elapsed_all_greedy = pd.DataFrame(columns=columns)

errors_encountered_all = []


p2 = mp.Pool(n_procs)
r = [p2.apply_async(get_regret_for_specific_dataset_greedy, args=[train_index, test_index, version]) 
          for train_index, test_index in loo.split(error_tensor)]
p2.close()
p2.join()    
    
for item in r:
    regret_all_greedy = regret_all_greedy.append(item.get()[0])
    idx_to_sample_all_greedy = idx_to_sample_all_greedy.append(item.get()[1])
    elapsed_all_greedy = elapsed_all_greedy.append(item.get()[2])

test index: [1]
test index: [3]
test index: [2]
test index: [0]
test index: [8]
test index: [12]
test index: [7]
test index: [4]
test index: [17]
test index: [16]
test index: [10]
test index: [5]
test index: [6]
test index: [9]
test index: [13]
test index: [15]
test index: [14]
test index: [11]
test index: [47]
test index: [19]
test index: [22]
test index: [32]
test index: [48]
test index: [18]
test index: [20]
test index: [38]
test index: [27]
test index: [43]
test index: [52]
test index: [41]
test index: [64]
test index: [62]
test index: [51]
test index: [45]
test index: [63]
test index: [67]
test index: [25]
test index: [56]
test index: [50]
test index: [68]
test index: [78]
test index: [36]
test index: [66]
test index: [111]
test index: [79]
test index: [69]
test index: [81]
test index: [97]
test index: [89]
test index: [30]
test index: [31]
test index: [34]
test index: [73]
test index: [75]
test index: [29]
test index: [92]
test index: [93]
test index: [72]
test index: [83]
test i

Process ForkPoolWorker-522:
Process ForkPoolWorker-540:
Process ForkPoolWorker-494:
Process ForkPoolWorker-502:
Process ForkPoolWorker-532:
Process ForkPoolWorker-498:
Process ForkPoolWorker-517:
Process ForkPoolWorker-549:
Process ForkPoolWorker-515:
Process ForkPoolWorker-527:
Process ForkPoolWorker-543:
Process ForkPoolWorker-504:
Process ForkPoolWorker-542:
Process ForkPoolWorker-510:


KeyboardInterrupt: 

Process ForkPoolWorker-534:
Process ForkPoolWorker-509:
Process ForkPoolWorker-528:
Process ForkPoolWorker-558:
Process ForkPoolWorker-556:
Process ForkPoolWorker-501:
Process ForkPoolWorker-529:
Process ForkPoolWorker-507:
Process ForkPoolWorker-475:
Traceback (most recent call last):
  File "/data/cy438/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/data/cy438/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/data/cy438/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "<ipython-inp

  File "/data/cy438/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/data/cy438/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/data/cy438/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/data/cy438/anaconda3/lib/python3.7/site-packages/numpy/lib/npyio.py", line 447, in load
    pickle_kwargs=pickle_kwargs)
  File "/data/cy438/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/data/cy438/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/data/cy438/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/data/cy438/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/data/cy438/anaconda3/lib/python

  File "<ipython-input-88-a12be6fd9c9a>", line 38, in get_regret_for_specific_dataset_greedy
    error_tensor = np.load(os.path.join(input_path, 'error_tensor.npy'))
  File "<ipython-input-88-a12be6fd9c9a>", line 38, in get_regret_for_specific_dataset_greedy
    error_tensor = np.load(os.path.join(input_path, 'error_tensor.npy'))
  File "<ipython-input-88-a12be6fd9c9a>", line 38, in get_regret_for_specific_dataset_greedy
    error_tensor = np.load(os.path.join(input_path, 'error_tensor.npy'))
  File "<ipython-input-88-a12be6fd9c9a>", line 38, in get_regret_for_specific_dataset_greedy
    error_tensor = np.load(os.path.join(input_path, 'error_tensor.npy'))
  File "<ipython-input-88-a12be6fd9c9a>", line 38, in get_regret_for_specific_dataset_greedy
    error_tensor = np.load(os.path.join(input_path, 'error_tensor.npy'))
  File "<ipython-input-88-a12be6fd9c9a>", line 38, in get_regret_for_specific_dataset_greedy
    error_tensor = np.load(os.path.join(input_path, 'error_tensor.npy'))
  Fi

  File "/data/cy438/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/data/cy438/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-88-a12be6fd9c9a>", line 39, in get_regret_for_specific_dataset_greedy
    runtime_tensor = np.load(os.path.join(input_path, 'runtime_tensor.npy'))
  File "<ipython-input-88-a12be6fd9c9a>", line 39, in get_regret_for_specific_dataset_greedy
    runtime_tensor = np.load(os.path.join(input_path, 'runtime_tensor.npy'))
  File "/data/cy438/anaconda3/lib/python3.7/site-packages/numpy/lib/npyio.py", line 447, in load
    pickle_kwargs=pickle_kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
  File "/data/cy438/anaconda3/lib/python3.7/site-packages/numpy/lib/npyio.py", line 447, in load
    pickle_kwargs=pickle_kwargs)
  File "/data/cy438/anaconda3/lib/python3.7/site-packages/numpy/lib/

KeyboardInterrupt
  File "/data/cy438/anaconda3/lib/python3.7/site-packages/numpy/lib/format.py", line 712, in read_array
    array = numpy.fromfile(fp, dtype=dtype, count=count)
  File "/data/cy438/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-88-a12be6fd9c9a>", line 39, in get_regret_for_specific_dataset_greedy
    runtime_tensor = np.load(os.path.join(input_path, 'runtime_tensor.npy'))
  File "/data/cy438/anaconda3/lib/python3.7/site-packages/numpy/lib/format.py", line 712, in read_array
    array = numpy.fromfile(fp, dtype=dtype, count=count)
KeyboardInterrupt
  File "<ipython-input-88-a12be6fd9c9a>", line 39, in get_regret_for_specific_dataset_greedy
    runtime_tensor = np.load(os.path.join(input_path, 'runtime_tensor.npy'))
  File "/data/cy438/anaconda3/lib/python3.7/site-packages/numpy/lib/npyio.py", line 447, in load
    pickle_kwargs=pickle_kwargs)
KeyboardInterrupt
  File "/data/cy438/ana

  File "/data/cy438/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-88-a12be6fd9c9a>", line 39, in get_regret_for_specific_dataset_greedy
    runtime_tensor = np.load(os.path.join(input_path, 'runtime_tensor.npy'))
  File "/data/cy438/anaconda3/lib/python3.7/site-packages/numpy/lib/npyio.py", line 447, in load
    pickle_kwargs=pickle_kwargs)
Process ForkPoolWorker-548:
  File "/data/cy438/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
Process ForkPoolWorker-541:
Process ForkPoolWorker-514:
Process ForkPoolWorker-505:
  File "/data/cy438/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
Process ForkPoolWorker-555:
  File "<ipython-input-88-a12be6fd9c9a>", line 39, in get_regret_for_specific_dataset_greedy
    runtime_tensor = np.load(os.path.join(input_path, 'runtime_tensor.npy'

  File "<ipython-input-88-a12be6fd9c9a>", line 39, in get_regret_for_specific_dataset_greedy
    runtime_tensor = np.load(os.path.join(input_path, 'runtime_tensor.npy'))
  File "/data/cy438/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/data/cy438/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-88-a12be6fd9c9a>", line 39, in get_regret_for_specific_dataset_greedy
    runtime_tensor = np.load(os.path.join(input_path, 'runtime_tensor.npy'))
  File "/data/cy438/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
Process ForkPoolWorker-524:
  File "/data/cy438/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-88-a12be6fd9c9a>", line 39, in get_regret_for_specific_dataset_greedy
    runtime_ten